In [1]:
#Imports
import cv2
import face_recognition

#function w path to image 
def get_face_encoding(image_path):
    image = face_recognition.load_image_file(image_path)
    encodings = face_recognition.face_encodings(image)
    if len(encodings) > 0:
        return encodings[0]  
    else:
        print(f"No face found in {image_path}")
        return None

# Load reference images and get the face encodings using face_reg library
#Reference 1 - Minh
reference_face_encoding_1 = get_face_encoding("reference1.jpg")
#Reference 2 - Vishnu
reference_face_encoding_2 = get_face_encoding("reference2.jpg")
#Reference 3 - Varun
reference_face_encoding_3 = get_face_encoding("reference3.jpg")

# Store the encodings and names in a list if a face was found
known_face_encodings = []
known_face_names = []

#name matching face references compared with reference_face_encoding
if reference_face_encoding_1 is not None:
    known_face_encodings.append(reference_face_encoding_1)
    known_face_names.append("Minh Nguyen") 

if reference_face_encoding_2 is not None:
    known_face_encodings.append(reference_face_encoding_2)
    known_face_names.append("Vishnu") 

if reference_face_encoding_3 is not None:
    known_face_encodings.append(reference_face_encoding_3)
    known_face_names.append("Varun")  

# Check if any faces were loaded before proceeding
if len(known_face_encodings) == 0:
    print("No reference faces loaded. Exiting...")
    exit()

# Initialize video capture
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

while True:
    ret, frame = cap.read()

    if ret:
        # Convert the frame from BGR to RGB 
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Find all face locations and encodings in the current frame
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            # Compare the face encodings with the known faces in reference_face_encoding
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)

            name = "No Match!"

            # If a match is found, use the corresponding name
            if True in matches:
                match_index = matches.index(True)
                name = known_face_names[match_index]

            # Draw rectangles around the detected faces
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)

            # Display the name of the recognized person
            cv2.putText(frame, name, (left + 6, bottom - 6), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # Display the video feed
        cv2.imshow("video", frame)

    key = cv2.waitKey(1)
    if key == ord("q"):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
